In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [41]:
num_input = 3
num_output = 3

num_hidden1 = 16
num_hidden2 = 16

my_rmse_list = []

In [3]:

session = tf.Session()

x = tf.placeholder(tf.float32, shape=[None, num_input])

y_true = tf.placeholder(tf.float32, shape=[None, num_output], name='y_true')
y_pred = tf.placeholder(tf.float32, shape=[None, num_output])


In [4]:
data_csv = pd.read_csv('../myCSVData.csv')
X = data_csv[['in1','in2','in3']]
y = data_csv[['out1','out2','out3']]
#X, y
print (type(X), type(y))
#y

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [5]:

X_a = X.values
y_a = y.values

In [6]:

def cost_func(layer, my_y_true):
    ret = tf.sqrt(tf.reduce_mean(tf.square(layer - my_y_true)))
    #print (layer , my_y_true)
    #ret = tf.reduce_mean(tf.square(layer))
    return ret

In [69]:
def nn_model_fn(features, labels, mode):
    global my_rmse_list
    # Neural Network Structure
    # Input Layer
    input_layer = tf.reshape(features["x"],[-1, 3]) 
    
    # Hidden Layer #1
    h1 = tf.layers.dense(inputs=input_layer, 
                         units=num_hidden1,
                         use_bias=True,
                         activation=tf.nn.relu)

    # Hidden Layer #2
    h2 = tf.layers.dense(inputs=h1, 
                         units=num_hidden2,
                         use_bias=True,
                         activation=tf.nn.relu)

    # Output Layer
    output_layer = tf.layers.dense(inputs=h2, 
                         units=num_output,
                         use_bias=True,
                         activation=None)
    predictions = {
        "output" : output_layer
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    cost = cost_func(output_layer, labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
        
        train_op = optimizer.minimize(
                    loss=cost,
                    global_step=tf.train.get_global_step())
        #my_rmse_list += [cost.eval]
        return tf.estimator.EstimatorSpec(mode=mode, loss=cost, train_op=train_op)
    
    eval_metric_ops = {
        "error":tf.metrics.mean_squared_error(
        labels=labels, predictions=predictions["output"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=cost, eval_metric_ops=eval_metric_ops)

In [70]:
my_nn_esti = tf.estimator.Estimator(model_fn=nn_model_fn, model_dir="./sav")

tensors_to_log = {"RMSE": "output"}
rmse_list = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './sav', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffa5b43cbe0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [71]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    #x={"in1" : X_a[:,0], "in2":X_a[:,1], "in3":X_a[:,2]},
                    x={"x":X_a},
                    y=y_a,
                    batch_size=100,
                    num_epochs=None,
                    shuffle=True)
my_nn_esti.train(
                input_fn=train_input_fn,
                steps=1000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./sav/model.ckpt-26010
INFO:tensorflow:Saving checkpoints for 26011 into ./sav/model.ckpt.
INFO:tensorflow:loss = 0.1390894910483121, step = 26011
INFO:tensorflow:global_step/sec: 497.228
INFO:tensorflow:loss = 0.047273734989341606, step = 26111 (0.208 sec)
INFO:tensorflow:global_step/sec: 468.204
INFO:tensorflow:loss = 0.0354991219766615, step = 26211 (0.216 sec)
INFO:tensorflow:global_step/sec: 406.597
INFO:tensorflow:loss = 0.03864107985568689, step = 26311 (0.244 sec)
INFO:tensorflow:global_step/sec: 492.253
INFO:tensorflow:loss = 0.10624254941605757, step = 26411 (0.204 sec)
INFO:tensorflow:global_step/sec: 489.293
INFO:tensorflow:loss = 0.0510398218393486, step = 26511 (0.204 sec)
INFO:tensorflow:global_step/sec: 489.471
INFO:tensorflow:loss = 0.031078293385511848, step = 26611 (0.204 sec)
INFO:tensorflow:global_step/sec: 478.048
INFO:tensorflow:loss = 0.044179311438411445, step = 26711 (0.208 s

In [73]:
print (my_rmse_list)

[<tf.Tensor 'Sqrt:0' shape=() dtype=float64>, <tf.Tensor 'Sqrt:0' shape=() dtype=float64>, <bound method Tensor.eval of <tf.Tensor 'Sqrt:0' shape=() dtype=float64>>]
